# Setup

In [7]:
import torch
from torch import nn
# from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
from tqdm import tqdm
import json

from torchmetrics import Accuracy

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score

In [8]:
from IPython.display import clear_output
import os, sys, shutil
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
# from tqdm import tqdm, trange
from glob import glob
import random

import cv2 as cv

In [9]:
print(os.getcwd())

/home/linhpika/git/HeartResearch/Experiment/Approach/EfficientB0


In [10]:
for i in range(3):
    os.chdir("..")

print(os.getcwd())

/home/linhpika/git/HeartResearch


In [28]:
# D:\GitCloneProject\HeartResearch\Data set\v4_data\med_scaleogram_h256_w512_seglen1600_scl500
main_data_dir = os.getcwd() + "/Data set"

label_csv_path = main_data_dir + "/Label_img"

# lead = ['I','II','III','aVR','aVL','aVF','V1','V2','V3','V4','V5','V6']
lead = [0,1,2,3,4,5,6,7,8,9,10,11]
lead_to_onehot = {leads : torch.nn.functional.one_hot(torch.tensor([index])[0], num_classes = 12) for index, leads in enumerate(lead)}

class_la = [1,2,3,4,5,6,7,8,9]
class_to_onehot = {classes : torch.nn.functional.one_hot(torch.tensor([index])[0], num_classes = 9) for index, classes in enumerate(class_la)}

img_data_dir = "/media/mountHDD1/ecg/med_scaleogram_h256_w512_seglen1600_scl500"
img_data_list =  glob(img_data_dir + "/*")

print(len(os.listdir(img_data_dir)))

425820


In [12]:
print(img_data_list[0])

/media/mountHDD1/ecg/med_scaleogram_h256_w512_seglen1600_scl500/A0118_lead7_seg3.png


In [29]:
label_df = pd.read_csv(label_csv_path)
label_df.head()

,Unnamed: 0,Image,Class,Lead
0,0,A0118_lead7_seg3,3,7
1,1,A3997_lead9_seg5,7,9
2,2,A2161_lead2_seg7,5,2
3,3,A4685_lead1_seg13,5,1
4,4,A1999_lead0_seg4,6,0


In [9]:
label_df["Class"][0]

4

In [16]:
print(os.listdir(main_data_dir))

['Label.csv', 'README.md', 'Label_img']


 # Data Loader

In [19]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torch.nn.functional import one_hot

In [20]:
torch.cuda.is_available()

True

In [30]:
ratio = [0.8, 0.1, 0.1]

train_index = int(len(img_data_list)*ratio[0])
valid_index = int(len(img_data_list)*(ratio[0]+ratio[1]))

train_image_paths = img_data_list[:train_index]
valid_image_paths = img_data_list[train_index:valid_index]
test_image_paths = img_data_list[valid_index:]

train_label = label_df.iloc[:train_index,:]
valid_label = label_df.iloc[train_index:valid_index,:]
test_label = label_df.iloc[valid_index:,:]

print(len(train_image_paths))
print(len(valid_image_paths))
print(len(test_image_paths))

print(train_label.shape)
print(valid_label.shape)
print(test_label.shape)

340656
42582
42582
(340656, 4)
(42582, 4)
(42582, 4)


In [22]:
train_image_paths[0]

'/media/mountHDD1/ecg/med_scaleogram_h256_w512_seglen1600_scl500/A0118_lead7_seg3.png'

In [23]:
train_label

,Unnamed: 0,Image,Class,Lead
0,0,A0118_lead7_seg3,3,7
1,1,A3997_lead9_seg5,7,9
2,2,A2161_lead2_seg7,5,2
3,3,A4685_lead1_seg13,5,1
4,4,A1999_lead0_seg4,6,0
...,...,...,...,...
340651,340651,A5061_lead4_seg3,5,4
340652,340652,A6644_lead3_seg2,3,3
340653,340653,A3898_lead10_seg9,1,10
340654,340654,A5733_lead4_seg6,5,4


In [31]:
class HeartData(Dataset):
    def __init__(self, label_df, data_path):
        self.label_df = label_df
        self.data_path = data_path
        
#         self.onehot_label_class = one_hot(self.label_df['Class'])
#         self.onehot_label_lead = one_hot(self.label_df['Lead'])
    def __len__(self):
        return len(self.data_path)
        
    def __getitem__(self, index):
#         imgs = []
#         labels = []
#         labels.append(self.onehot_label_class)
#         labels.append(self.onehot_label_lead)
        class_label = self.label_df['Class'][index]
        class_label = class_to_onehot[class_label]
    
        lead_label = self.label_df['Lead'][index]
        lead_label = lead_to_onehot[lead_label]

        data_img = cv.imread(self.data_path[index])
        torch_img = torch.from_numpy(data_img).permute(-1, 0, 1)
        
#         lead_label = self.onehot_label_lead[index]
#         class_label = self.onehot_label_class[index]
        
        return lead_label, class_label, torch_img

In [16]:
class_label = train_label['Class'][0]
class_label = class_to_onehot[class_label]
print(type(class_label))
lead_label = train_label['Lead'][0]
lead_label = lead_to_onehot[lead_label]
print(type(lead_label))
data_img = cv.imread(train_image_paths[1])
torch_img = torch.from_numpy(data_img).permute(-1, 0, 1)
print(type(torch_img))
print(torch_img.shape)

<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
torch.Size([3, 253, 506])


In [32]:
train_dataset = HeartData(label_df, train_image_paths)
valid_dataset = HeartData(label_df, valid_image_paths)
test_dataset = HeartData(label_df, test_image_paths)

In [26]:
len(valid_dataset)

42582

In [33]:
valid_dataset[5]


(tensor([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]),
 tensor([0, 0, 0, 0, 0, 0, 0, 1, 0]),
 tensor([[[128, 128, 128,  ..., 128, 128, 128],
          [128, 128, 128,  ..., 132, 164, 182],
          [128, 128, 128,  ..., 137, 146, 187],
          ...,
          [254, 248, 251,  ..., 251, 160, 219],
          [251, 251, 255,  ..., 248, 167, 254],
          [248, 251, 255,  ..., 251, 180, 244]],
 
         [[  0,   0,   0,  ...,   0,   0,   0],
          [  0,   0,   0,  ...,   0,   0,   0],
          [  0,   0,   0,  ...,   0,   0,   0],
          ...,
          [221, 229, 225,  ..., 225, 255, 255],
          [225, 225, 213,  ..., 229, 255, 221],
          [229, 225, 197,  ..., 225, 255, 232]],
 
         [[  0,   0,   0,  ...,   0,   0,   0],
          [  0,   0,   0,  ...,   0,   0,   0],
          [  0,   0,   0,  ...,   0,   0,   0],
          ...,
          [  0,   0,   0,  ...,   0,  86,  28],
          [  0,   0,   0,  ...,   0,  80,   0],
          [  0,   0,   0,  ...,   0,  67,   2]]]

In [34]:
train_dataloader = DataLoader(train_dataset, batch_size = 16, shuffle = True, pin_memory = True)
valid_dataloader = DataLoader(valid_dataset, batch_size = 16, shuffle = True, pin_memory = True)
test_dataloader = DataLoader(test_dataset, batch_size = 1, shuffle = True, pin_memory = True)

In [21]:
len(train_dataloader)

1098

In [22]:
# multi-task learning - efficient B0
# data loader -> 2 labels: lead + disease
# Model: 2 output: 1 vector for 12 leads (softmax) + 1 vector for disease
# loss funct: loss lead + loss class => backward
# random choice: notice: seed(python, numpy, torch) same
        
# multi-channel - efficient B2 - quite similar to video classification

# Model 

In [35]:
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
from torch import nn

ori_model = efficientnet_b0(weights = EfficientNet_B0_Weights.IMAGENET1K_V1)

Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-3dd342df.pth" to /home/linhpika/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-3dd342df.pth
100.0%


In [24]:
# ori_model

In [36]:
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
from torch import nn

class HeartModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.ori_model = efficientnet_b0(weights = EfficientNet_B0_Weights.IMAGENET1K_V1)
        del self.ori_model.classifier
        self.ori_model.classifier = nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(1280, 21),
            nn.Softmax(dim = 1)
        )
        
    def forward(self, x):        
        logits = self.ori_model(x)
        
        return (logits[:, :12], logits[:, 12:])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = HeartModel().to(device)
print(model)

HeartModel(
  (ori_model): EfficientNet(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): SiLU(inplace=True)
      )
      (1): Sequential(
        (0): MBConv(
          (block): Sequential(
            (0): Conv2dNormActivation(
              (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
              (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): SiLU(inplace=True)
            )
            (1): SqueezeExcitation(
              (avgpool): AdaptiveAvgPool2d(output_size=1)
              (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
              (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
              (activation): SiLU(inplace=True)
              (scale_

# Training

In [43]:
from datetime import datetime

opt_mapping = {
    "Adam" : torch.optim.Adam
}

loss_mapping = {
    "CCE" : nn.CrossEntropyLoss()
}

class Training:
    def __init__(self, 
                 device: str = "cpu",
                 learning_rate:float = 0.0001,
                 optimizer:str = "Adam",
                 loss:str = "CCE",
                 model = model,
#                  batchsize:int = 32,
                 epochs:int = 100,
#                  label_df: pd.DataFrame = label_df, 
#                  root_dir: str = img_data_dir, 
#                  ratio: tuple = (0.8, 0.1, 0.1),
#                  resize: tuple = None, #  (256, 512)
#                  seed: int = 777
                ):
        
        # Setup
        self.device = device
        self.model = model
        self.model.to(self.device)
        self.lr = learning_rate
        self.optimizer = opt_mapping[optimizer](self.model.parameters(), lr=self.lr)
        self.loss_fn = loss_mapping[loss]
        self.ep = epochs
        self.lead_accuracy = Accuracy(task="multiclass", num_classes=12).to(self.device)
        self.cls_accuracy = Accuracy(task="multiclass", num_classes=9).to(self.device)
        
        # Data
        self.train_data = train_dataloader
        self.valid_data = valid_dataloader
        self.test_data = test_dataloader
    

    
    def __update__(self):
        train_losses = []
        val_losses = []
        for e in range(self.ep):
            train_class_acc = 0
            train_lead_acc = 0
            batch_cnt = 0
            self.model.train()
            for batch, (y1, y2, X) in tqdm(enumerate(self.train_data)):
                batch_cnt = batch
                y1, y2 = y1.to(self.device), y2.to(self.device)
                pred1, pred2 = self.model((X/255).to(self.device))
                l1 = self.loss_fn(pred1, y1.to(self.device, dtype = torch.float))
                l2 = self.loss_fn(pred2, y2.to(self.device, dtype = torch.float))
                train_loss = l1 + l2

                
                self.optimizer.zero_grad()
                train_loss.backward()
                self.optimizer.step()
                train_lead_acc += self.lead_accuracy(torch.argmax(pred1, dim = 1), torch.argmax(y1, dim = 1)).item()
                train_class_acc += self.cls_accuracy(torch.argmax(pred2, dim = 1), torch.argmax(y2, dim = 1)).item()
                        
            mean_train_cls_acc = train_class_acc/(batch_cnt + 1)
            mean_train_led_acc = train_lead_acc/(batch_cnt + 1)
            # Show train_loss, train_acc, val_loss, val_acc
            if e%10 == 0: 
                print(f"Epoch: {e} - Train Loss: {train_loss.item()} - Train class acc: {mean_train_cls_acc} - Train lead acc: {mean_train_led_acc}")
#             " - Val Loss: {val_loss.item()} - Val Acc: {val_acc.item()}")
            train_losses.append(train_loss)
            self.validation()
        loss_plot(self, train_losses = train_losses, val_losses = val_losses, n_epochs = e + 1, check_folder = 'run_efficientB0')
        self.evaluation()
        
    def validation(self):
        self.model.eval()
        valid_class_acc = 0
        valid_lead_acc = 0
        batch_cnt = 0
        
        with torch.no_grad():
            valid_class_acc = 0
            valid_lead_acc = 0
            batch_cnt = 0
            for batch, (y1, y2, X) in tqdm(enumerate(self.valid_data)):
                batch_cnt = batch

                y1, y2 = y1.to(self.device), y2.to(self.device)
                pred1, pred2 = self.model((X/255).to(self.device))

                l1 = self.loss_fn(pred1, y1.to(self.device, dtype = torch.float))
                l2 = self.loss_fn(pred2, y2.to(self.device, dtype = torch.float))
                val_loss = l1 + l2
                

                valid_lead_acc += self.lead_accuracy(torch.argmax(pred1, dim = 1), torch.argmax(y1, dim = 1)).item()
                valid_class_acc += self.cls_accuracy(torch.argmax(pred2, dim = 1), torch.argmax(y2, dim = 1)).item()                

            mean_valid_cls_acc = valid_class_acc/(batch_cnt + 1)
            mean_valid_led_acc = valid_lead_acc/(batch_cnt + 1)
            val_losses.append(val_loss)
            self.checkpoint(valid_lead_acc = mean_valid_cls_acc, 
                       valid_class_acc = mean_valid_led_acc, 
                       val_total_loss = val_loss,
                       epoch = epoch, 
                       model = self.model,
                       optimizer = self.optimizer,
                       check_folder = 'run_efficientB0')

            print(f"Val_loss: {val_loss.item()} - Val class acc: {mean_valid_cls_acc} - Val lead acc: {mean_valid_led_acc}")
        
        return val_loss, mean_valid_cls_acc, mean_valid_led_acc

    def loss_plot(self, train_losses, val_losses, n_epochs, check_folder):
        now = datetime.now().strftime("%m-%d-%Y - %H-%M-%S")
        run_dir = os.getcwd() + f"/{check_folder}"
        if not os.path.exists(run_dir):
            os.mkdir(run_dir)
        save_dir = run_dir + f"/{now}"
        if not os.path.exists(save_dir):
            os.mkdir(save_dir)
        save_loss_dir = save_dir + "\save_losses"
        if not os.path.exists(save_loss_dir):
            os.mkdir(save_loss_dir)
        save_fig_losses = os.path.join(save_loss_dir, 'plot_losses.png')  
        
        plt.figure()
        plt.plot(range(1, n_epochs + 1), train_losses, label='Train Loss')
        plt.plot(range(1, n_epochs + 1), val_losses, label='Validation Loss')
        plt.xlabel('Epoch')
        # plt.xticks()
        plt.ylabel('Loss')
        plt.title('Training and Validation Loss')
        plt.legend()
        plt.savefig(save_fig_losses)
    
    def checkpoint(self, valid_lead_acc, valid_class_acc, 
                   val_total_loss,
                   epoch, 
                   model,
                   optimizer,
                   check_folder
#                    logs
                  ):
        old_valid_lead_acc = 0
        old_valid_class_acc = 0
        old_valid_loss = 1e23
        if valid_lead_acc >= old_valid_lead_acc and valid_class_acc >= old_valid_class_acc and val_total_loss <= old_valid_loss:
            old_valid_lead_acc = valid_lead_acc
            old_valid_class_acc = valid_class_acc
            old_valid_loss = val_total_loss
            save_dict = {
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': val_total_loss,
                'val_lead_acc': valid_lead_acc,
                'val_class_acc': valid_class_acc
            }
            
         # Saving best model
        now = datetime.now().strftime("%m-%d-%Y - %H-%M-%S")
        run_dir = os.getcwd() + f"/{check_folder}"
        if not os.path.exists(run_dir):
            os.mkdir(run_dir)
        save_dir = run_dir + f"/{now}"
        if not os.path.exists(save_dir):
            os.mkdir(save_dir)
        save_best_model_dir = save_dir + "\save_best_model"
        if not os.path.exists(save_best_model_dir):
            os.mkdir(save_best_model_dir)
        save_best_model_path = save_best_model_dir + f"/{save_dict['loss']:>7f}_{save_dict['val_lead_acc']:>7f}_{save_dict['val_class_acc']:>7f}_{now}.pt"
        torch.save(save_dict, save_best_model_path)

#         config_path = save_dir + '/config.json'
#         with open(config_path, mode='w') as file:
#             json.dump(vars(args), file)
    
    def evaluation(self):
        
        self.model.eval()
        
        y1_eval = [] 
        y2_eval = []
        pred1_eval = []
        pred2_eval = []
        
        with torch.no_grad():
            for batch, (y1, y2, X) in tqdm(enumerate(self.test_data)):
                batch_cnt = batch
                
                y1, y2 = y1.to(self.device), y2.to(self.device)
                pred1, pred2 = self.model((X/255).to(self.device))
                
                y1_eval.append(torch.argmax(y1, dim=1)[0])
                y2_eval.append(torch.argmax(y2, dim=1)[0])
                pred1_eval.append(torch.argmax(pred1, dim=1)[0])
                pred2_eval.append(torch.argmax(pred2, dim=1)[0])
                
            y1_eval = torch.stack(y1_eval)
            y2_eval = torch.stack(y2_eval)
            pred1_eval = torch.stack(pred1_eval)
            pred2_eval = torch.stack(pred2_eval)
                
            pred1 = torch.nn.functional.one_hot(pred1_eval, num_classes = 12)
            pred2 = torch.nn.functional.one_hot(pred2_eval, num_classes = 9)
            y1 = torch.nn.functional.one_hot(y1_eval, num_classes = 12)
            y2 = torch.nn.functional.one_hot(y2_eval, num_classes = 9)
#             print(y1)
#             print(y2)
#             print(pred1)
#             print(pred2)
#             print(y2_eval)
#             print(pred2_eval)
#             print(torch.argmax(y1, dim=1).cpu().numpy()[0].tolist())
#             print(torch.argmax(y1, dim=1).cpu().numpy().tolist())
            
            # Classification report
            for i in range (len(class_la)):
                class_la[i] = str(class_la[i])

            # Classification report
            print(classification_report(
                torch.argmax(y1, dim=1).cpu().numpy().tolist(), 
                torch.argmax(pred1, dim=1).cpu().numpy().tolist(),
                target_names=lead))    
            

            print(classification_report(
                torch.argmax(y2, dim=1).cpu().numpy().tolist(), 
                torch.argmax(pred2, dim=1).cpu().numpy().tolist(), 
                target_names=class_la)) 

            # Confusion Matrix
            ConfusionMatrixDisplay.from_predictions(torch.argmax(y1, dim=1).cpu().numpy().tolist(), torch.argmax(pred1, dim=1).cpu().numpy().tolist(), cmap = "PuBuGn")
            plt.show()

            ConfusionMatrixDisplay.from_predictions(torch.argmax(y2, dim=1).cpu().numpy().tolist(), torch.argmax(pred2, dim=1).cpu().numpy().tolist(), cmap = "PuBuGn")
            plt.show()

            # ROC Curve
            for i in range(12):
                r1 = roc_auc_score(y1[:, i].cpu(), pred1[:, i].cpu())
                print("The ROC AUC score of "+ lead[i] +" is: "+str(r1))

            for i in range(9):
                r2 = roc_auc_score(y2[:, i].cpu(), pred2[:, i].cpu())
                print("The ROC AUC score of "+ class_la[i] +" is: "+str(r2))

            # Compute ROC curve and ROC area for each lead
            lead_fpr = {}
            lead_tpr = {}
            roc_auc = {}
            lead_roc_auc = dict()
            for i in range(12):
                lead_fpr[i], lead_tpr[i], _ = roc_curve(y1[:, i].cpu(), pred1[:, i].cpu(), drop_intermediate=False)
                roc_auc[i] = auc(lead_fpr[i], lead_tpr[i])

            plt.plot(lead_fpr[0], lead_tpr[0],'turquoise',label=f'I: ROC curve of {lead[0]} (area = %0.2f)' % roc_auc[0])
            plt.plot(lead_fpr[1], lead_tpr[1],'peachpuff',label=f'II: ROC curve of {lead[1]} (area = %0.2f)' % roc_auc[1])
            plt.plot(lead_fpr[2], lead_tpr[2],'paleturquoise',label=f'III: ROC curve of {lead[2]} (area = %0.2f)' % roc_auc[2])
            plt.plot(lead_fpr[3], lead_tpr[3],'pink',label=f'aVR: ROC curve of {lead[3]} (area = %0.2f)' % roc_auc[3])
            plt.plot(lead_fpr[4], lead_tpr[4],'lightcoral',label=f'aVL: ROC curve of {lead[4]} (area = %0.2f)' % roc_auc[4])
            plt.plot(lead_fpr[5], lead_tpr[5],'peachpuff',label=f'aVF: ROC curve of {lead[5]} (area = %0.2f)' % roc_auc[5])
            plt.plot(lead_fpr[6], lead_tpr[6],'steelblue',label=f'V1: ROC curve of {lead[6]} (area = %0.2f)' % roc_auc[6])
            plt.plot(lead_fpr[7], lead_tpr[7],'forestgreen',label=f'V2: ROC curve of {lead[7]} (area = %0.2f)' % roc_auc[7])
            plt.plot(lead_fpr[8], lead_tpr[8],'darkslategray',label=f'V3: ROC curve of {lead[8]} (area = %0.2f)' % roc_auc[8])
            plt.plot(lead_fpr[9], lead_tpr[9],'orange',label=f'V4: ROC curve of {lead[9]} (area = %0.2f)' % roc_auc[9])
            plt.plot(lead_fpr[10], lead_tpr[10],'maroon',label=f'V5: ROC curve of {lead[10]} (area = %0.2f)' % roc_auc[10])
            plt.plot(lead_fpr[11], lead_tpr[11],'navy',label=f'V6: ROC curve of {lead[11]} (area = %0.2f)' % roc_auc[11])

            plt.plot([0, 1], [0, 1], 'k--')
            plt.xlim([-0.1, 1.1])
            plt.ylim([-0.1, 1.1])
            plt.xlabel('False Positive Rate')
            plt.ylabel('True Positive Rate')
            plt.title('Receiver operating characteristic of class')
            plt.legend(loc="lower right")
            plt.show()

            # Compute ROC curve and ROC area for each class
            class_fpr = {}
            class_tpr = {}
            class_roc_auc = dict()
            for i in range(9):
                class_fpr[i], class_tpr[i], _ = roc_curve(y2[:, i].cpu(), pred2[:, i].cpu(), drop_intermediate=False)
                roc_auc[i] = auc(class_fpr[i], class_tpr[i])

            plt.plot(class_fpr[0], class_tpr[0],'turquoise',label='1: ROC curve of class 1 (area = %0.2f)' % roc_auc[0])
            plt.plot(class_fpr[1], class_tpr[1],'peachpuff',label='2: ROC curve of class 2 (area = %0.2f)' % roc_auc[1])
            plt.plot(class_fpr[2], class_tpr[2],'paleturquoise',label='3: ROC curve of class 3 (area = %0.2f)' % roc_auc[2])
            plt.plot(class_fpr[3], class_tpr[3],'pink',label='4: ROC curve of class 4 (area = %0.2f)' % roc_auc[3])
            plt.plot(class_fpr[4], class_tpr[4],'lightcoral',label='5: ROC curve of class 5 (area = %0.2f)' % roc_auc[4])
            plt.plot(class_fpr[5], class_tpr[5],'peachpuff',label='6: ROC curve of class 6 (area = %0.2f)' % roc_auc[5])
            plt.plot(class_fpr[6], class_tpr[6],'steelblue',label='7: ROC curve of class 7 (area = %0.2f)' % roc_auc[6])
            plt.plot(class_fpr[7], class_tpr[7],'forestgreen',label='8: ROC curve of class 8 (area = %0.2f)' % roc_auc[6])
            plt.plot(class_fpr[8], class_tpr[8],'darkslategray',label='9: ROC curve of class 9 (area = %0.2f)' % roc_auc[6])

            plt.plot([0, 1], [0, 1], 'k--')
            plt.xlim([-0.1, 1.1])
            plt.ylim([-0.1, 1.1])
            plt.xlabel('False Positive Rate')
            plt.ylabel('True Positive Rate')
            plt.title('Receiver operating characteristic of lead')
            plt.legend(loc="lower right")
            plt.show()          

In [ ]:
# Metadata

# Training Setup
device = "cuda"
learning_rate = 0.0001
optimizer = "Adam"
loss = "CCE"
# batchsize = 128
epochs = 100
# ratio = (0.8, 0.1, 0.1)

# Data

monitor = Training(device = device,
                   learning_rate = learning_rate,
                   optimizer = optimizer,
                   loss = loss,
                   model = model,
#                    batchsize = batchsize,
                   epochs = epochs)
#                    label_df = label_df, 
#                    root_dir = img_data_dir, 
#                    ratio = ratio)
monitor.__update__()

240it [01:12,  3.44it/s]

In [28]:
# Unitest

# monitor.get_sample_count()

In [29]:
# monitor.update()

# Evaluation

In [30]:
# monitor.evaluation()

# Clear Buffer

In [31]:
# monitor.clear_buffer()